<a href="https://colab.research.google.com/github/ymoslem/OpenNMT-Tutorial/blob/main/2-NMT-Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install OpenNMT-py 3.x
!pip3 install OpenNMT-py

# Prepare Your Datasets
Please make sure you have completed the [first exercise](https://colab.research.google.com/drive/1rsFPnAQu9-_A6e2Aw9JYK3C8mXx9djsF?usp=sharing).

In [ ]:
# Open the folder where you saved your prepapred datasets from the first exercise
# You might need to mount your Google Drive first
%cd /content/drive/MyDrive/nmt/
!ls

# Create the Training Configuration File

The following config file matches most of the recommended values for the Transformer model [Vaswani et al., 2017](https://arxiv.org/abs/1706.03762). As the current dataset is small, we reduced the following values: 
* `train_steps` - for datasets with a few millions of sentences, consider using a value between 100000 and 200000, or more! Enabling the option `early_stopping` can help stop the training when there is no considerable improvement.
* `valid_steps` - 10000 can be good if the value `train_steps` is big enough. 
* `warmup_steps` - obviously, its value must be less than `train_steps`. Try 4000 and 8000 values.

Refer to [OpenNMT-py training parameters](https://opennmt.net/OpenNMT-py/options/train.html) for more details. If you are interested in further explanation of the Transformer model, you can check this article, [Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/).

In [31]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: UN.en-fr.fr-filtered.fr.subword.train
        path_tgt: UN.en-fr.en-filtered.en.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: UN.en-fr.fr-filtered.fr.subword.dev
        path_tgt: UN.en-fr.en-filtered.en.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.fren

# Stop training if it does not imporve after n validations
early_stopping: 4

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps 
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 3000

# Default: 10000 - Run validation after n steps
valid_steps: 1000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 2
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
# [Optional] Check the content of the configuration file
!cat config.yaml

# Build Vocabulary

For large datasets, it is not feasable to use all words/tokens found in the corpus. Instead, a specific set of vocabulary is extracted from the training dataset, usually betweeen 32k and 100k words. This is the main purpose of the vocabulary building step.

In [7]:
# Find the number of CPUs/cores on the machine
!nproc --all

2


In [25]:
# Build Vocabulary

# -config: path to your config.yaml file
# -n_sample: use -1 to build vocabulary on all the segment in the training dataset
# -num_threads: change it to match the number of CPUs to run it faster

!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-11-19 20:00:16,415 INFO] Counter vocab from -1 samples.
[2022-11-19 20:00:16,415 INFO] n_sample=-1: Build vocab on full datasets.
[2022-11-19 20:00:18,967 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=2138)

[2022-11-19 20:00:18,976 INFO] * Transform statistics for corpus_1(50.00%):
			* FilterTooLongStats(filtered=2032)

[2022-11-19 20:00:19,035 INFO] Counters src:14705
[2022-11-19 20:00:19,035 INFO] Counters tgt:11884


From the **Runtime menu** > **Change runtime type**, make sure that the "**Hardware accelerator**" is "**GPU**".


In [12]:
# Check if the GPU is active
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1759f39f-df0c-a03f-3066-463f5fec7c38)


In [24]:
# Check if the GPU is visable to PyTorch

import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

True
Tesla T4
Free GPU memory: 15007.75 out of: 15109.75


# Training

Now, start training your NMT model! 🎉 🎉 🎉

In [32]:
!rm -rf drive/MyDrive/nmt/models/

In [ ]:
# Train the NMT model
!onmt_train -config config.yaml

In [ ]:
# For error debugging try:
# !dmesg -T

# Translation

Translation Options:
* `-model` - specify the last model checkpoint name; try testing the quality of multiple checkpoints
* `-src` - the subworded test dataset, source file
* `-output` - give any file name to the new translation output file
* `-gpu` - GPU ID, usually 0 if you have one GPU. Otherwise, it will translate on CPU, which would be slower.
* `-min_length` - [optional] to avoid empty translations
* `-verbose` - [optional] if you want to print translations

Refer to [OpenNMT-py translation options](https://opennmt.net/OpenNMT-py/options/translate.html) for more details.

In [35]:
# Translate the "subworded" source file of the test dataset
# Change the model name, if needed.
!onmt_translate -model models/model.fren_step_3000.pt -src UN.en-fr.fr-filtered.fr.subword.test -output UN.en.translated -gpu 0 -min_length 1

[2022-11-19 21:00:48,673 INFO] PRED SCORE: -0.2185, PRED PPL: 1.24 NB SENTENCES: 2000


In [36]:
# Check the first 5 lines of the translation file
!head -n 5 UN.en.translated

▁ 1 1 . ▁Reaffirms ▁the ▁commitments ▁made ▁at ▁the ▁Fourth ▁M inisterial ▁Conference ▁of ▁the ▁World ▁Trade ▁Organization , ▁held ▁in ▁Doha , ▁and ▁the ▁Thir d ▁Unit ed ▁Nations ▁Conference ▁on ▁the ▁Lea st ▁Developed ▁Countries , ▁held ▁in ▁Brussels ▁from ▁ 1 4 ▁to ▁ 2 0 ▁May ▁ 2 0 0 1 , See ▁A / CONF . 1 9 1 / 1 1 1 ▁and ▁ 1 2 . ▁and ▁in ▁this ▁regard ▁calls ▁upon ▁developed ▁countries ▁that ▁have ▁not ▁ye t ▁done ▁so ▁to ▁work ▁together ▁to ▁work ▁towards ▁the ▁objective ▁of ▁duty - free ▁and ▁quota - free ▁market ▁access ▁for ▁all ▁least ▁developed ▁countries , ▁and ▁also ▁notes ▁that ▁the ▁proposals ▁for ▁developing ▁countries ▁would ▁be ▁helpful ;
▁ 1 0 . ▁Recognizes , ▁where ▁appropriate , ▁international ▁cooperation ▁in ▁the ▁field ▁of ▁critical ▁information ▁infrastructures , ▁including ▁the ▁establishment ▁of ▁emergency ▁information ▁and ▁coordination ▁mechanisms , ▁as ▁well ▁as ▁the ▁sharing ▁of ▁experience , ▁information ▁on ▁and ▁ te lecommunications , ▁response ▁and ▁sol

In [38]:
# If needed install/update sentencepiece
!pip3 install --upgrade -q sentencepiece

# Desubword the translation file
!python3 MT-Preparation/subwording/3-desubword.py target.model UN.en.translated

     |████████████████████████████████| 1.3 MB 15.7 MB/s 
Done desubwording! Output: UN.en.translated.desubword


In [39]:
# Check the first 5 lines of the desubworded translation file
!head -n 5 UN.en.translated.desubword

11. Reaffirms the commitments made at the Fourth Ministerial Conference of the World Trade Organization, held in Doha, and the Third United Nations Conference on the Least Developed Countries, held in Brussels from 14 to 20 May 2001,See A/CONF.191/111 and 12. and in this regard calls upon developed countries that have not yet done so to work together to work towards the objective of duty-free and quota-free market access for all least developed countries, and also notes that the proposals for developing countries would be helpful;
10. Recognizes, where appropriate, international cooperation in the field of critical information infrastructures, including the establishment of emergency information and coordination mechanisms, as well as the sharing of experience, information on and telecommunications, response and solutions to such incidents, and the identification of incidents in accordance with domestic law.
Recalling its relevant resolutions, including resolution 58/292 of 6 May 2004,

In [40]:
# Desubword the target file (reference) of the test dataset
# Note: You might as well have split files *before* subwording during dataset preperation, 
# but sometimes datasets have tokeniztion issues, so this way you are sure the file is really untokenized.
!python3 MT-Preparation/subwording/3-desubword.py target.model UN.en-fr.en-filtered.en.subword.test

Done desubwording! Output: UN.en-fr.en-filtered.en.subword.test.desubword


In [42]:
# Check the first 5 lines of the desubworded reference
!head -n 5 UN.en-fr.en-filtered.en.subword.test.desubword

11. Reaffirms the commitments made at the Fourth Ministerial Conference of the World Trade Organization held at Doha and at the Third United Nations Conference on the Least Developed Countries, held at Brussels from 14 to 20 May 2001,See A/CONF.191/11 and 12. and in this regard calls upon developed countries that have not already done so to work towards the objective of duty-free, quota-free market access for all least developed countries' exports, and notes that consideration of proposals for developing countries to contribute to improved market access for least developed countries would also be helpful;
10. Engage in international cooperation, when appropriate, to secure critical information infrastructures, including by developing and coordinating emergency warning systems, sharing and analysing information regarding vulnerabilities, threats and incidents and coordinating investigations of attacks on such infrastructures in accordance with domestic laws.
Recalling its relevant resol

# MT Evaluation

There are several MT Evaluation metrics such as BLEU, TER, METEOR, COMET, BERTScore, among others.

Here we are using BLEU. Files must be detokenized/desubworded beforehand.

In [ ]:
# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

In [ ]:
# Install sacrebleu
!pip3 install sacrebleu

In [45]:
# Evaluate the translation (without subwording)
!python3 compute-bleu.py UN.en-fr.en-filtered.en.subword.test.desubword UN.en.translated.desubword

Reference 1st sentence: 11. Reaffirms the commitments made at the Fourth Ministerial Conference of the World Trade Organization held at Doha and at the Third United Nations Conference on the Least Developed Countries, held at Brussels from 14 to 20 May 2001,See A/CONF.191/11 and 12. and in this regard calls upon developed countries that have not already done so to work towards the objective of duty-free, quota-free market access for all least developed countries' exports, and notes that consideration of proposals for developing countries to contribute to improved market access for least developed countries would also be helpful;
MTed 1st sentence: 11. Reaffirms the commitments made at the Fourth Ministerial Conference of the World Trade Organization, held in Doha, and the Third United Nations Conference on the Least Developed Countries, held in Brussels from 14 to 20 May 2001,See A/CONF.191/111 and 12. and in this regard calls upon developed countries that have not yet done so to work 

# More Features and Directions to Explore

Experiment with the following ideas:
* Icrease `train_steps` and see to what extent new checkpoints provide better translation, in terms of both BLEU and your human evaluation.

* Check other MT Evaluation mentrics other than BLEU such as [TER](https://github.com/mjpost/sacrebleu#ter), [WER](https://blog.machinetranslation.io/compute-wer-score/), [METEOR](https://blog.machinetranslation.io/compute-bleu-score/#meteor), [COMET](https://github.com/Unbabel/COMET), and [BERTScore](https://github.com/Tiiiger/bert_score). What are the conceptual differences between them? Is there special cases for using a specific metric?

* Continue training from the last model checkpoint using the `-train_from` option, only if the training stopped and you want to continue it. In this case, `train_steps` in the config file should be larger than the steps of the last checkpoint you train from.
```
!onmt_train -config config.yaml -train_from models/model.fren_step_3000.pt
```

* **Ensemble Decoding:** During translation, instead of adding one model/checkpoint to the `-model` argument, add multiple checkpoints. For example, try the two last checkpoints. Does it improve quality of translation? Does it affect translation seepd?

* **Averaging Models:** Try to average multiple models into one model using the [average_models.py](https://github.com/OpenNMT/OpenNMT-py/blob/master/onmt/bin/average_models.py) script, and see how this affects translation quality.
```
python3 average_models.py -models model_step_xxx.pt model_step_yyy.pt -output model_avg.pt
```
* **Release the model:** Try this command and see how it reduce the model size.
```
onmt_release_model --model "model.pt" --output "model_released.pt
```
* **Use CTranslate2:** For efficient translation, consider using [CTranslate2](https://github.com/OpenNMT/CTranslate2), a fast inference engine. Check out an [example](https://gist.github.com/ymoslem/60e1d1dc44fe006f67e130b6ad703c4b).

* **Work on low-resource languages:** Find out more details about [how to train NMT models for low-resource languages](https://blog.machinetranslation.io/low-resource-nmt/).

* **Train a multilingual model:** Find out helpful notes about [training multilingual models](https://blog.machinetranslation.io/multilingual-nmt).

* **Publish a demo:** Show off your work through a [simple demo with CTranslate2 and Streamlit](https://blog.machinetranslation.io/nmt-web-interface/).
